In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
estados = ['https://www.fapema.br/category/editais/editais-em-aberto/', 'https://www.fapema.br/category/editais/editais-encerrados/']

fapema = pd.DataFrame()
titles_proyects = []
links_proyects = []

for estado in estados:
 
    parser_estado = parser(estado)

    ult_page = parser_estado.xpath('//a[@class="page-numbers"]/text()')[-1]
    pages = []
    for i in range(1, int(ult_page)+1):
        page = estado + 'page/' + str(i) + '/'
        pages.append(page)

    for p in pages:
        parser_page = parser(p)

        title_proyects_parciales = parser_page.xpath('//div[@class="item-content"]//h3/div/a/text()')

        links_proyects_parciales = parser_page.xpath('//div[@class="item-content"]//h3/div/a/@href')
    
        titles_proyects += title_proyects_parciales
        links_proyects += links_proyects_parciales

fapema['Títulos'] = titles_proyects
fapema['links'] = links_proyects

/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fapema.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
pdfs_proyects = []
description_proyects = []

for link in links_proyects:

    parser_proyect = parser(link)
    try:
        pdf_proyect = parser_proyect.xpath('//div[@class="pdfjs-fullscreen"]/a/@href')[0].split('&')[0]
    except:
        pdf_proyect=''

    descripcion_proyect_bruta = parser_proyect.xpath('//p/text()')
    descripcion_proyect_bruta2 = parser_proyect.xpath('//tr/td//text()')

    descripcion_proyect = ''

    for d in descripcion_proyect_bruta:
        d = d.strip()
        descripcion_proyect = descripcion_proyect + d + ' '

    for d in descripcion_proyect_bruta2:
        d = d.strip()
        if d != '':
            descripcion_proyect = descripcion_proyect + d + ' '


    pdfs_proyects.append(descripcion_proyect.strip(' '))
    description_proyects.append(descripcion_proyect)

print(len(pdfs_proyects))
print(len(description_proyects))

136
136


In [5]:
fapema['PDFs'] = pdfs_proyects
fapema['Descripción'] = description_proyects

In [6]:
fapema.to_excel('Brasil_FAPEMA.xlsx')